<a href="https://colab.research.google.com/github/rweidinger/dogapp/blob/dev/GPU_Puppy_Party.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Net for Dog Image Recognition

## Environment Setup

Import libraries and mount Drive

In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

'2.0.0'

In [34]:
import os
import pandas as pd
import numpy as np
from sklearn import neural_network
import scipy.io as sio
import tarfile
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.strings import to_number
import matplotlib.pyplot as plt
import random as rn
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import re

ModuleNotFoundError: No module named 'tensorflow.strings'

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

## Import train and test data

In [5]:
# Define function to download .mat files
def loadMatFile(file_name):
  root_path = 'data/'
  return sio.loadmat(root_path + file_name)

In [6]:
# Import train and test sets
train_dict = loadMatFile('train_data.mat')
test_dict = loadMatFile('test_data.mat')
train_data = train_dict['train_data']
test_data = test_dict['test_data']

In [38]:
def extract_labels(regex_pattern, dictionary, return_type):
    label_list = []
    if 'train' in '{}'.format(dictionary):
        prefix = 'train'
        for array in dictionary['{}_info'.format(prefix)][0][0][0]:
            label_list.append(re.search(regex_pattern, array[0][0]).group(1))
    elif 'test' in '{}'.format(dictionary):
        prefix = 'test'
        for array in dictionary['{}_info'.format(prefix)][0][0][0]:
            label_list.append(re.search(regex_pattern, array[0][0]).group(1))
    else:
        print('Error: please check your input dictionary. You may have entered the name of a non-existant dictionary.')
    print('Processed {} {} labels!'.format(len(label_list), prefix))
    if return_type == 'series':
        return pd.Series(label_list)
    elif return_type == 'array':
        return np.asarray(label_list)
    elif return_type == 'list':
        return label_list

train_labels = extract_labels(regex_pattern = '-(.*?)/n', dictionary = train_dict, return_type = 'series')
test_labels = extract_labels(regex_pattern = '-(.*?)/n', dictionary = test_dict, return_type = 'series')

Processed 12000 train labels!
Processed 8580 test labels!


In [41]:
test_labels_numeric, uniques = test_labels.factorize()
train_labels_numeric, uniques = train_labels.factorize()
test_labels_numeric

array([  0,   0,   0, ..., 119, 119, 119])

In [57]:
image_count = int(input('Enter the number of images you would like to train on: '))
#y_binary = to_categorical(train_labels)

#image_count = 200
model = Sequential()
model.add(Dense(2000, activation = 'relu', input_shape = (12000,)))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(500, activation = 'relu'))
model.add(Dense(120, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_data[:image_count], train_labels_numeric[:image_count], validation_split = 0.2, epochs = 100)

Train on 9600 samples, validate on 2400 samples
Epoch 1/10
9600/9600 [==============================] - 24s 3ms/sample - loss: 4.6280 - accuracy: 0.0171 - val_loss: 8.8074 - val_accuracy: 0.0000e+00
Epoch 2/10
9600/9600 [==============================] - 22s 2ms/sample - loss: 4.4423 - accuracy: 0.0219 - val_loss: 9.9029 - val_accuracy: 0.0000e+00
Epoch 3/10
9600/9600 [==============================] - 23s 2ms/sample - loss: 4.3737 - accuracy: 0.0252 - val_loss: 9.8318 - val_accuracy: 0.0000e+00
Epoch 4/10
9600/9600 [==============================] - 21s 2ms/sample - loss: 4.2986 - accuracy: 0.0324 - val_loss: 13.3706 - val_accuracy: 0.0000e+00
Epoch 5/10
9600/9600 [==============================] - 21s 2ms/sample - loss: 4.2200 - accuracy: 0.0367 - val_loss: 12.2186 - val_accuracy: 0.0000e+00
Epoch 6/10
9600/9600 [==============================] - 20s 2ms/sample - loss: 4.1677 - accuracy: 0.0409 - val_loss: 13.6573 - val_accuracy: 0.0000e+00
Epoch 7/10
9600/9600 [=====================

In [48]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 10)                120010    
_________________________________________________________________
dense_22 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_23 (Dense)             (None, 120)               1320      
Total params: 121,440
Trainable params: 121,440
Non-trainable params: 0
_________________________________________________________________


## Exploratory Data Analysis

In [11]:
train_dict

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Wed Mar 28 12:03:37 2012',
 '__version__': '1.0',
 '__globals__': [],
 'train_info': array([[(array([[array(['n02085620-Chihuahua/n02085620_5927.jpg'], dtype='<U38')],
        [array(['n02085620-Chihuahua/n02085620_4441.jpg'], dtype='<U38')],
        [array(['n02085620-Chihuahua/n02085620_1502.jpg'], dtype='<U38')],
        ...,
        [array(['n02116738-African_hunting_dog/n02116738_6754.jpg'], dtype='<U48')],
        [array(['n02116738-African_hunting_dog/n02116738_9333.jpg'], dtype='<U48')],
        [array(['n02116738-African_hunting_dog/n02116738_2503.jpg'], dtype='<U48')]],
       dtype=object), array([[array(['n02085620-Chihuahua/n02085620_5927'], dtype='<U34')],
        [array(['n02085620-Chihuahua/n02085620_4441'], dtype='<U34')],
        [array(['n02085620-Chihuahua/n02085620_1502'], dtype='<U34')],
        ...,
        [array(['n02116738-African_hunting_dog/n02116738_6754'], dtype='<U44')],
        [array([

In [13]:
train_data.shape

(12000, 12000)

In [14]:
test_data.shape

(8580, 12000)